# 要約 
このJupyter Notebookは、Kaggleの「20の質問」ゲームコンペティションに向けたプレイヤーエージェントの構築に関するもので、具体的には自然言語処理における未知のエンティティの特定に挑んでいます。主な焦点は、Qwen 2 7b - Instruct モデルを使用して、質問者（"asker"）、回答者（"answerer"）、および推測者（"guesser"）の役割を果たすエージェントを開発することです。

### 問題
このノートブックは、言語モデルが限られた質問数でターゲットとなるキーワードを特定するという、推論と効率的な情報収集を必要とする複雑な問題に取り組んでいます。また、他のエージェントとの協力が必要なため、戦略的に質問・回答を生成しなければなりません。

### 手法とライブラリ
1. **モデルの選定**: メインモデルとしてQwen 2 7b - Instructが使用されていますが、Hugging Faceの他のモデルに切り替えることも可能です。
2. **テキスト生成**: Hugging Face Transformersライブラリを用いて、LLM（大規模言語モデル）を使用し、質問や推測を生成します。また、モデルの量子化設定を行い、GPUメモリの制約を考慮しています。
3. **関数設計**:
   - `ask()`: 質問者エージェントが過去の履歴に基づいてロジカルな質問を生成します。
   - `answer()`: 最も頻繁に出現する回答を集計して提供します。
   - `guess()`: ゲームの履歴をもとに推測を生成します。
4. **エージェントの初期化**: ノートブックでエージェントを初期化し、Kaggle環境に適応させるためのコードも含まれています。
5. **デバッグ機能**: デバッグモードを有効にすることで、生成されたプロンプトや応答を観察し、モデルの改善に役立てることができます。

### 改善点
ノートブックは、このエージェントの性能を向上させるためのいくつかの提案（別のLLMモデルの追加、ゲーム状態トラッキング、推測精度の向上など）を示しています。また、在宅トレーニングデータセットを利用してモデルをファインチューニングすることも考慮されています。

このように、ノートブックは「20の質問」ゲームをプレイするための高度にカスタマイズされた言語モデルの作成を目指しており、実際の競技環境での適用を可能にしています。

---


# 用語概説 
以下に、Jupyter Notebookに登場する専門用語について、初心者がつまずきそうなものの解説を示します。一般に知られているものや、大学で学ぶ程度のレベルに達している基礎的な内容は省いています。

### 1. **ハードコーディング**
プログラムにおいて、データや設定をコード内に直接書き込むこと。柔軟性がなく、変更するにはコードを修正する必要があるため、設定ファイルからの読み込みに比べて保守性が低い。

### 2. **量子化 (Quantization)**
深層学習モデルのパラメータや計算を、より少ないビット数で表す手法。これはモデルのサイズを小さくし、計算効率を上げるために使用される。特에より、ストレージやメモリの節約が可能となり、特に限られたリソース環境（例：モバイルデバイスやIoTデバイス）でのモデル運用に役立つ。

### 3. **BitsAndBytes**
特にGPUでの計算時に精度を下げつつも計算を可能にするためのライブラリ。低ビット数で計算を行い、省メモリで動作することを可能にします。この手法を使うことで、メモリの使用効率を向上させながらモデルを実行できます。

### 4. **メモリエラー (Memory Error)**
プログラムが必要とするメモリを確保できない場合に発生するエラー。特に大きなモデルを扱うときに注意が必要で、メモリ管理が適切でないと発生します。このようなエラーは、通常、メモリを節約するための戦略を見直すか、ハードウェアのアップグレードを考慮する必要があります。

### 5. **システムプロンプト (System Prompt)**
LLM（大規模言語モデル）に対して与える指示や文脈を設定するためのテキスト。LLMが生成する応答の方向やスタイルを決定付ける要素となる。

### 6. **多段階プロンプティング (Multi-Step Prompting)**
複数のステップで可能な応答や質問を生成するプロセス。これにより、より複雑な質問や応答が可能になり、エージェントの推論能力が向上する。

### 7. **observe関数**
Kaggle環境からの入力データを処理し、そのデータに基づいてエージェントの行動を決定する関数。この関数がどのように環境に応じた応答を決定しているのかを理解することがエージェントの戦略を考える上で重要です。

### 8. **次元 (Dimension)**
しばしばデータの特徴や要素の数を指し、モデルの複雑さや表現可能な情報の量に関わります。次元が高いということは、モデルはより多くの情報を保持できますが、一方で計算リソースの負荷も増加します。

これらの用語は、特に実務経験の少ない初心者にとって理解が難しい場合があるため、精一杯簡潔に解説しました。ただし、実際の使用や背景にはもっと深い情報や理解が必要な場合があります。それぞれの用語に関して興味があれば、さらなるリソースを用いて調べることをお勧めします。

---


# はじめに
このノートブックは提出可能な状態であり、改善戦略や考慮すべきアイデアについて詳しい指示が含まれています。モデルは Qwen 2 7b - Instruct に設定されていますが、他の huggingface のモデルに変更することも可能です。

このノートブックに変更を加えなければ、すべてのセルを実行するか、ノートブックをコミットして競技に提出できます。

## このノートブックはどのようなスコアを得るでしょうか？

これに対する答えは難しいです。キーワード検索の範囲は広大です。未知のエンティティの特定は言語モデルにとって難しい問題であり、さらにこのコンペティションは、他のエージェント対チームと同じゲームで協力するという複雑性があります。この 2x2 の設定は、様々なゲーム結果をもたらす挑戦的な文脈を作り出します。これについては多くのディスカッション投稿が作成されています。さらに、キーワードリストに依存して推測を生成するオーバーフィットモデルもあります。これは公共リーダーボードでポイントを獲得しますが、エージェントに秘密のキーワードリストが与えられた場合には失敗します。このため、公共リーダーボードはモデルの性能の良い基準ではない可能性があります。モデルの性能を自己評価で判断することをお勧めします。
1. 質問は論理的に見えますか？
2. 推測は質問に従っていますか？
3. あなたのエージェントは質問に適切に回答していますか？
4. 変更を加えた場合、あなたのモデルは以前よりも良くなっていますか？

これらの質問に対する答えは、スコアやメダルを与えるものではありませんが、私たちが解決しようとしている本質的な問題の核心にあります：

**小型の言語モデルを設計し、未知のエンティティ特定ゲームを他の言語モデルと一緒にプレイし、うまく行うことができるようにすること。**

## 提出スクリプト

以下のスクリプトは、20の質問ゲーム用に Kaggle 環境内で実行される .py ファイルを作成します。これは、各ラウンドでテキストを生成するために LLM（以下でダウンロード）の機能を利用します。スクリプトには、ゲーム環境内で与えられた構造と変数と対話するクラスが含まれています。また、LLM に質問者、回答者、推測者の役割を果たさせるための 3 つの関数も含まれています。最後に、初期化されたモデルとトークナイザーを使用したテキスト生成、およびゲームからの適切なエージェント呼び出しが含まれています。 

### 戦略

このスクリプトで採用されている戦略には以下が含まれています：
* ゲームを開始し、それがどのように進むべきかを LLM に示すための「ハードコーディング」された最初の質問。
* ゲームの履歴を見直し、応答を生成する質問者および推測者エージェント。
* 5 回ループして最も頻繁な応答を返す回答者エージェント。
* 競技制限をシミュレートするために cuda:0 にモデルをロードします。

このスクリプトには含まれていないもの：
* 公共リーダーボードでのパフォーマンスを向上させるために LLM にキーワードを供給すること。
* 推測者が検索するための堅牢なキーワードリストを提供するための外部語彙リストを供給すること。

## 改善の余地
* より堅牢な LLM モデルを追加する。huggingface のオープン LLM リーダーボードで ~7B またはそれ以下のモデルを確認し、スクリプトにロードを試みる。
* Phi3、Gemma、または LLaMa 3 を試し、Gwen と比較してどのようにパフォーマンスするかを確認する。（提出フォルダにライブラリをダウンロードするのを忘れないでください！）
* 大きなモデルを追加する場合は、bitsandbytes と accelerate のインストール/インポートを行い、量子化設定を 4 ビットにしてロードします。
* 提供された答えに基づいて、キーワードがどのカテゴリに属する可能性が高いかを定義するゲーム状態トラッキングを追加する。
* ゲームの初期段階の幅広い質問から、後半段階の特定の質問へ移動するためのゲームフェーズトラッキングを追加する。
* 直肢応答生成ではなく、マルチステッププロンプティングを追加する。
* テキスト生成パラメータ（温度など）を実験する。
* 指示調整された Qwen モデルは、時々マルチレベルプロンプトに苦戦することに気づくでしょう。他のモデルは自己対話をより良く行うでしょうか？
* ファインチューニングされたモデル。標準のモデルは、20の質問に特化して作られていないため、いくつかは優れています。進行中の 20 の質問コンペティションからの Q&A ペアのデータセットが役立つかもしれません。

## よくある質問
* メモリエラーが出ている？工場出荷時リセットを行い、スクリプトを再実行してください。モデルを二重にロードした可能性があります！
* 質問者や推測者からの応答が正しくフォーマットされていないのはなぜですか？レスポンスをパースする創造的な方法を見つける必要があります。これは、プロンプティングを通じて行うことも、生成されたテキストから有効な応答を抽出するためのパーサーをハードコーディングすることもできます。

# さあ、コードに進みましょう

### 最初に - 依存関係のインストール
スクリプトのためのいくつかの依存関係をインストールし、提出にアップロードされる一時ディレクトリにインストールします。これは、スクリプトが Kaggle 環境で動作することを保証するために不可欠です。
他のパッケージをインストールする必要がある場合は、以下の両方のセルに追加してください。

In [ ]:
# 必要なライブラリのインストール
!pip install accelerate bitsandbytes  # accelerateとbitsandbytesライブラリをインストールします。
# accelerateは、モデルのトレーニングや推論を高速化するためのライブラリで、bitsandbytesは低精度の計算を可能にするためのライブラリです。これらをインストールすることで、Kaggle環境でのモデルの動作を最適化します。

このセルは少し時間がかかります。

In [ ]:
import os  # osモジュールをインポートします。このモジュールはシステム機能にアクセスするために使用されます。

# 特定のディレクトリに依存関係をインストールする
os.system("pip install -t /tmp/submission/lib accelerate bitsandbytes")  
# このコマンドは、'accelerate'と'bitsandbytes'ライブラリを'/tmp/submission/lib'ディレクトリにインストールします。
# これは、Kaggle環境内でスクリプトが正常に動作するために必要です。

### スナップショットのダウンロード: このセルは何をしますか？

このセクションでは、モデルをダウンロードして一時ディレクトリに保存し、そのディレクトリが圧縮されて提出フォルダに含まれます。提出後にスクリプトによって呼び出されます。ゲーム環境はKaggle環境で実行されるため、モデルの重みをダウンロードして提出する必要があります。このセルは読み込みに数分かかります。

注意: これはモデルやトークナイザーをロードするものではありません。そのステップはメインスクリプトで行われます。

現在のところ、モデルは Qwen 2 7b Instruct に設定されています。

In [ ]:
from huggingface_hub import snapshot_download  # Hugging Face Hubからモデルをダウンロードするための関数をインポートします。
from pathlib import Path  # ファイルパスの操作のためのPathモジュールをインポートします。
import shutil  # ファイルやディレクトリの操作のためのshutilモジュールをインポートします。

# モデルを保存するためのパスを定義します。
model_path = Path("/tmp/submission/")

# すでに同じパスが存在する場合は削除します。
if model_path.exists():
    shutil.rmtree(model_path)  # model_pathが存在する場合、そのディレクトリを削除します。

# 新しいパスを作成します。
model_path.mkdir(parents=True)  # model_pathを作成します。必要に応じて親ディレクトリも作成します。

# 指定されたリポジトリからモデルをダウンロードします。
snapshot_download(
    repo_id= "Qwen/Qwen2-7B-Instruct",  # ダウンロードするモデルのリポジトリID
    local_dir=model_path  # モデルを保存するローカルディレクトリ
)  # この関数は指定されたリポジトリからモデルをダウンロードし、指定したローカルディレクトリに保存します。

# メインスクリプト

ノートブックの最上部にあるマジックコマンドは、Kaggle環境内で実行され、コンペティションに提出される .py ファイルを作成します。

- このスクリプトを評価のために実行すると、量子化されたモデルがGPUメモリにロードされます。Gwen 2 7bは大きいため、4ビット量子化されたバージョンがロードされ、利用可能な15GBのうち7.4GBのメモリを占有します。T4での動作に制限があります。

In [ ]:
%%writefile main.py

# 上記の行を評価セッションを実行する場合はコメントアウトしてください。

import os  # osモジュールをインポートします。

# このスクリプトがどの環境で実行されているかを確認します。これにより、異なる環境で提出や評価が可能になります。
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if not os.path.exists(KAGGLE_AGENT_PATH):
    KAGGLE_AGENT_PATH = "/tmp/submission/"

import sys
import itertools
import re
import torch
import typing as t
from pathlib import Path
from pydantic import BaseModel, Field, field_validator
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from IPython.display import display, Markdown

model_initialized = False
device = "cuda:0" if torch.cuda.is_available() else "cpu"  # GPUが利用できる場合はcuda:0を使用、そうでなければcpuを使用
t_dtype = torch.bfloat16
DEBUG = False

#==================量子化設定================
# Gwen 2 7bを量子化する必要があります。このモデルは、競技で使用される単一のT4 GPUに収まりません。
# これは、大きなモデルを量子化するための設定がどのように見えるかを参照できるように含まれています。
# また、モデルがすでにロードされているかを確認する条件文も展開されています。そうであれば、メモリ不足エラーを防ぐためにロードをスキップします。
# これはスクリプトの修正や評価セッションのチェック、新しいアイデアを試すために非常に役立ちます。

if 'model' in locals() and 'tokenizer' in locals() and model is not None and tokenizer is not None:
    print("モデルとトークナイザーはすでに存在します。作成をスキップします。")
else:
    quantization_config = BitsAndBytesConfig(load_in_4bit=True,  # 4ビットでモデルをロード
                                             bnb_4bit_compute_dtype=t_dtype)  # 量子化計算のデータ型を指定
    
    # 指定されたパスからモデルをロードします。
    model = AutoModelForCausalLM.from_pretrained(KAGGLE_AGENT_PATH, 
                                                 device_map=device, 
                                                 torch_dtype=t_dtype,
                                                 quantization_config=quantization_config)
    
    tokenizer = AutoTokenizer.from_pretrained(KAGGLE_AGENT_PATH, 
                                              torch_dtype=t_dtype)

        
#==================KAGGLE環境用の観察クラス================
# このクラスは、Kaggle環境からの観察入力を受け取り、スクリプト内で使用します。
# この観察クラスを提供してくださったTeam Riggingの公開ノートブックに感謝します。

class Observation(BaseModel):
    step: int
    role: t.Literal["guesser", "answerer"]
    turnType: t.Literal["ask", "answer", "guess"]
    keyword: str
    category: str
    questions: list[str]
    answers: list[str]
    guesses: list[str]
    
    @property
    def empty(self) -> bool:
        return all(len(t) == 0 for t in [self.questions, self.answers, self.guesses])
    
    def get_history(self) -> t.Iterator[tuple[str, str, str]]:
        return itertools.zip_longest(self.questions, self.answers, self.guesses, fillvalue="[none]")

    def get_history_text(self, *, skip_guesses: bool = False, skip_answer: bool = False, skip_question: bool = False) -> str:
        if not self.empty:
            history = "\n".join(
            f"""{'**質問:** ' + question if not skip_question else ''} -> {'**回答:** ' + answer if not skip_answer else ''}
            {'**前回の推測:** ' + guess if not skip_guesses else ''}
            """
            for i, (question, answer, guess) in enumerate(self.get_history())
            )
            return history
        return "まだありません。"


# =================システムプロンプト================
# ここでは、テキスト生成中にメッセージテンプレートに挿入されるシステムプロンプトを定義します。
# インストラクション/チャットモデルの場合、これはモデルが従うべきコンテキストと方向性を提供します。
# ここでは、さまざまなプロンプト戦略を展開できることがあり、調査することができます。
# 役割、ゲーム状態、またはゲームフェーズ（初期/中期/後期）に応じて異なるシステムプロンプトを持つこともできます。

system_prompt = """あなたは、インタラクティブな未知のエンティティ特定ゲーム「20の質問」をプレイしています。
あなたは全ての役割、質問者、回答者、推測者としての経験豊富なプレイヤーです。
あなたの目的は、秘匿キーワードをできるだけ少ない手数で推測し、勝利することです。"""

# =================質問者エージェント================
# 質問者エージェントは、まずキーワードが場所かどうかを尋ね、その後、赤道の北に位置するかを尋ねます。ここではLLMは呼び出されません。
# もし場所に関する質問の答えが「いいえ」の場合、非生物かどうかを尋ねます。
# これは、LLMにゲーム環境の少数ショットの例を提供し、最初のカテゴリーを二分するのに役立ちます。
# その後（ステップ > 1）において、過去の履歴に基づいて生成された質問を尋ねます。
# LLMを呼び出す前にさらなる条件付き質問を追加して、検索空間をさらに二分することもできます。
# 思考過程推論を開始するためにテキスト生成ステップを追加することもできます。
# max_new_tokensやtemperatureパラメータを確認してください。
# プロンプトの複雑さが増すにつれて、異なるパラメータを持つ異なるgenerate_text関数を作成したいと思うでしょう。
# たとえば、かなり高い新しいトークン制限と高い温度を持つgenerate_longなど。

def ask(observation: Observation) -> str:
    if observation.step == 0:
        return "秘匿キーワードは場所ですか？"
    elif observation.step == 1:
        if len(observation.answers) > 0 and observation.answers[0] == "yes":
            return "それは都市ですか？"
        else:
            return "それは生物ですか？"
    game_history = observation.get_history_text(skip_guesses=True)

    try:
        think = generate_text(f"""
        あなたは質問者として、次の質問をするところです。これまでにわかっていることは次のとおりです：

====== ゲーム履歴 =====
{game_history}
=========================

        上記の履歴に基づいて、次にどのような方向の質問をするべきか説明してください。
        """)
        
        parse = generate_text(f"""
        以下の現在の情報を確認し、次の最良の質問を述べてください。
        情報があまりない場合は、検索空間からカテゴリを排除するための「はい」または「いいえ」で答えられる質問をするのが最善です。  

        要約: "{think}""
        
        質問だけをしてください。質問と「？」以外のテキストを提供しないでください。
        """
        ).lower()
        if DEBUG:
            display(Markdown(f"### デバッグ: 質問者の思考:"))
            display(Markdown(think))
            display(Markdown(f"### デバッグ: 質問者の解析:"))
            display(Markdown(parse))
        return parse
    except Exception as e:
        print(f"質問をしている時のエラー: {e}")
        return 'それは場所ですか？'
    

# =================回答者エージェント================
# 5回ループして最も頻繁な応答を取ります。
# これはTeam Riggingノートブックで使用された技術です。
# 単一の応答よりも若干効果的です。

def answer(observation: Observation) -> t.Literal["yes", "no"]:
    if not observation.keyword:
        print("キーワードが回答者に提供されていません", file=sys.stderr)
        return "yes"
            
    last_question = observation.questions[-1]

    try:
        responses = []
        for i in range(5):
            response = generate_text(f"""
            20の質問ゲーム。このキーワードに対して「はい」または「いいえ」で答えます: {observation.keyword} 

            質問: "{last_question}"
            
            ルール:
            1. あなたのキーワード: {observation.keyword} と、そのカテゴリ: {observation.category} のみを考慮してください。
            2. 「はい」または「いいえ」とだけ答えてください。 その他は不要です。 
            あなたの答えはここに:
            """
            ).lower()
            if DEBUG:
                display(Markdown(f"### デバッグ: 回答者の応答:"))
                display(Markdown(f"デバッグ: {response}"))
            
            yes_no = re.findall(r'\b(yes|no)\b', response)
            if yes_no:
                responses.append(yes_no[0])

        if DEBUG:
            display(Markdown(f"デバッグ: 回答者のすべての応答は {responses} です。"))
        return max(set(responses), key=responses.count)

    except Exception as e:
        print(f"回答者でのエラー: {e}", file=sys.stderr)
        return "yes"

# ================推測者エージェント================
# このエージェントはゲームの履歴を使用して推測を行います。
# 一部のケースでは、LLMが関連語のループにハマってしまうため、これが役立たないことがあります。
# 異なるプロンプト戦略を試して、推測をスキップすることを含めてください。
# 解析戦略は、ゲームの応答が有効で理解可能であることを保証するのに役立ちます。

def guess(observation: Observation) -> str:
    game_history = observation.get_history_text(skip_guesses=False)
    
    think = generate_text(f"""
    あなたは現在推測者として、キーワードの情報をもとに推測を行います。 

== ゲーム履歴 ==
{game_history}
==================

    この情報に基づいて、秘匿キーワードに関連するカテゴリ1つと可能な推測を提示してください。
    あなたの推論を説明してください。
    """
    )
    
    parse = generate_text(f"""
    以下の応答を見直し、次の最良の推測を述べてください。
    あなたの情報があまりない場合は、特定の「野生な」推測をしてください。
    質問のように推測をしないでください。ただ、単語やフレーズをベースに推測してください。

    要約: "{think}"

    あなたの推測はここに:
    """
    ).lower()
    if DEBUG:
        print(f"ゲーム履歴: {game_history}")
        display(Markdown(f"### デバッグ: 推測者の思考:"))
        display(Markdown(think))
        display(Markdown(f"### デバッグ: 推測者の解析:"))
        display(Markdown(parse))
    
    return parse

#==============LLMメッセージ生成===============
# 修正したいパラメータがいくつかあります。
# 温度は生成されるテキストのランダムさを設定します。
# 低い値はより決定的、高い値はよりクリエイティブになります。
# max_new_tokensは生成されたテキストの長さを設定します。

    
def generate_text(prompt:str) -> str:
    sys_prompt = system_prompt
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": prompt},
    ]
    text = tokenizer.apply_chat_template(messages, 
                                         tokenize=False, 
                                         add_generation_prompt=True)
    
    inputs = tokenizer([text], return_tensors="pt").to(device)
    
    generated_ids = model.generate(inputs.input_ids,
                                   max_new_tokens=350, 
                                   do_sample=True, 
                                   temperature=0.1)  # これは生成されたテキストのランダムさ/創造性を設定します。低い値はより決定的です。
    
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, generated_ids)]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return response

#================KAGGLE環境用のエージェント初期化================
# 評価中にモデルに対して「入力」を変更するのは楽しいかもしれません！
# ボーナスとして、これによりファインチューニングデータセットを作成する可能性が開かれます！素晴らしいですね！
# 評価セッションをファインチューニングデータセット作成セッションに変更し、その出力をデータフレームに保存することもできるかもしれません。
# 可能性は無限ですが、時間は限られています。

agent = None

def observe(obs: t.Any) -> str:
    global agent
    observation = Observation(**obs.__dict__)

    try:
        match observation.turnType:
            case "ask":
                agent = ask(observation)
                return agent
            case "answer":
                agent = answer(observation)
                return agent
            case "guess":
                agent = guess(observation)
                return agent

            case _:
                raise ValueError("未知のターンタイプ")
    except Exception as e:
        print(str(e), file=sys.stderr)
        raise

def agent_fn(obs: t.Any, _: t.Any) -> str:
    return observe(obs)

# =========================TORCHバックエンドの有効化===========================
# このノートブックでスクリプトを実行しているときにエラーが発生することがあります。
# これらのバックエンド設定により、スクリプトがエラーなく実行されることが保証されます。やったね！

torch.backends.cuda.enable_mem_efficient_sdp(False)  # メモリ効率的なシステム動作を無効にします。
torch.backends.cuda.enable_flash_sdp(False)  # フラッシュスキップシステム動作を無効にします。

# 提出 - タールボール
これは、Kaggleが期待する形式に提出物をインストールし、圧縮します。モデル、スクリプト、および提出フォルダが圧縮されます。スクリプトにpipインストールを追加する場合は、それらを提出用のタールボールにも追加する必要があります。それらを/tmp/submission/libにインストールすることで、提出用のタールボールに追加します。

In [ ]:
!apt install pigz pv  # pigzは並列gzip圧縮を提供するユーティリティで、pvはデータの進行状況を表示するためのツールです。
# これらのツールは、提出物の圧縮を効率的に行うために使用されます。

In [ ]:
!tar --use-compress-program='pigz --fast' \  # pigzを使用して、圧縮を高速化します。
     -cf submission.tar.gz \  # 提出物をsubmission.tar.gzという名前で作成します。
     -C /kaggle/working main.py \  # /kaggle/workingディレクトリからmain.pyファイルを追加します。
     -C /tmp/submission/ \  # /tmp/submission/ディレクトリへ移動します。
     .  # その場所のすべてのファイルを圧縮します。
# このコマンドは、提出物の圧縮ファイルを作成し、Kaggleが期待する形式でパッケージングします。

# 評価
もしperform evalがTrueに設定されていれば、以下のコードはKaggleリポジトリから最新の公共キーワードをダウンロードし、ランダムなキーワードを生成します。これにより、Kaggle環境を初期化することを避け、メモリ不足エラーを引き起こすことを防ぎます。verboseデバッグモードを有効にするかどうか尋ねられます。これを有効にすると、LLMからのプロンプトや応答が表示されます。これはプロンプト構造を修正したり、プロンプト内のオブジェクトを呼び出す際、または出力がゲームの最終応答でないマルチステッププロンプト構造である場合に役立ちます。

* evalを実行する際は、アクセラレーターがオンになっていることを確認してください。
* evalセッションを実行する場合、プロンプト構造を変更してテストしたり、さまざまな戦略を試みることができます。この場合、LLM（モデルおよびトークナイザー）を初期化するスクリプトの最上部をコメントアウトすることをお勧めします。これを行わないと、毎回モデルが読み込まれ、メモリ切れになる可能性があります。
* 異なるモデルを試す際には、応答を得るのにかかる時間を確認してください。コンペティションは応答時間を60秒に制限しています！ （まずはGPUアクセラレーションを有効にしてください）。

### perform_eval？これは何のため？
変数 'perform_eval' を True に設定すると、すべてのセルを実行するかノートブックをコミットすると、エージェントが自分自身と対戦するシミュレートされた20の質問ゲームで評価されます。やった！

これをTrueに設定した場合、約2〜3分後にデバッグモードを行うかどうかの入力を求められます。

評価セッションをスキップして迅速にコミットと提出を行うためには、Falseに設定してください。
#### **迅速な評価のためにGPUアクセラレーションをオンにすることを忘れないでください。**

In [ ]:
perform_eval = False  # 評価セッションをスキップするためにperform_evalをFalseに設定します。これにより、迅速にコミットと提出を行うことができます。

In [ ]:
if perform_eval:
    import requests  # HTTPリクエストを行うためのrequestsモジュールをインポートします。
    import json  # JSONデータを扱うためのjsonモジュールをインポートします。
    import re  # 正規表現を扱うためのreモジュールをインポートします。
    import typing as t  # 型ヒントを使用するためのtypingモジュールをインポートします。
    import random  # ランダムな選択を行うためのrandomモジュールをインポートします。
    import time  # 時間関連の操作を行うためのtimeモジュールをインポートします。
    from IPython.display import display, Markdown  # 出力を表示するためのIPythonライブラリをインポートします。
    import signal  # 信号処理を行うためのsignalモジュールをインポートします。

    # ===============KAGGLE GITHUBから最新のキーワードを取得================
    # これはモデルがキーワードを探すためにトレーニングするものではありません。
    # これは評価に使用するためのリストを取得するだけです。
    
    url = "https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py"
    response = requests.get(url)  # 指定されたURLからリクエストを送信します。

    if response.status_code == 200:
        match = re.search(r'KEYWORDS_JSON = """(.*?)"""', response.text, re.DOTALL)  # キーワードのJSONを抽出します。
        if match:
            json_str = match.group(1)  # JSON文字列を取得します。
            keywords_dict = json.loads(json_str)  # JSONを辞書型に変換します。
        else:
            print("ファイル内にKEYWORDS_JSON変数が見つかりませんでした。")
    else:
        print("リクエストは以下のステータスコードで失敗しました:", response.status_code)

    def select_random_keyword(keywords_dict: t.List[dict]) -> str:
        category = random.choice(keywords_dict)  # ランダムにカテゴリを選択します。
        keyword_dict = random.choice(category['words'])  # 選択したカテゴリからランダムにキーワードを選びます。
        return keyword_dict['keyword'].lower()  # 小文字に変換して返します。
    
    # ===============20の質問評価セッション=====================
    def input_timeout(prompt, timeout):
        def timeout_handler(signum, frame):
            raise TimeoutError("指定時間後に入力がタイムアウトしました: {} 秒".format(timeout))

        signal.signal(signal.SIGALRM, timeout_handler)  # タイムアウトハンドラを設定
        signal.alarm(timeout)  # 指定した時間後にタイムアウトをトリガーします。

        try:
            user_input = input(prompt)  # プロンプトを表示し、ユーザー入力を待ちます。
            signal.alarm(0)  # タイムアウトをキャンセルします。
            return user_input
        except TimeoutError as e:
            print(e)
            return None

    timeout = 10  # タイムアウト時間を10秒に設定

    try:
        DEBUG_input = input_timeout("冗長デバッグモードを有効にしますか？ (y/n) [デフォルトはn] ", timeout)
        DEBUG = DEBUG_input.lower() == 'y' if DEBUG_input else False  # ユーザーの入力に基づいてDEBUGフラグを設定
    except:
        DEBUG = False
        print("入力が受信されなかったため、デフォルトでfalseに設定します。")

    class MockObservation:
        def __init__(self, step: int, role: str, turnType: str, keyword: str, category: str, questions: list[str], answers: list[str], guesses: list[str]):
            self.step = step  # ステップカウンタ
            self.role = role  # エージェントの役割（質問者、回答者など）
            self.turnType = turnType  # ターンの種類（ask, answer, guess）
            self.keyword = keyword  # 認識されるキーワード
            self.category = category  # カテゴリ名
            self.questions = questions  # 質問のリスト
            self.answers = answers  # 回答のリスト
            self.guesses = guesses  # 推測のリスト

    def test_20_questions():
        global DEBUG
        step = 0  # ステップを初期化
        role = "answerer"  # 初期役割を設定
        turnType = "ask"  # 初期ターンの種類を設定
        keyword = select_random_keyword(keywords_dict)  # キーワードをランダムに選択
        category = ""  # カテゴリは初期化
        questions = []  # 質問のリスト
        answers = []  # 回答のリスト
        guesses = []  # 推測のリスト
        display(Markdown("# 20の質問評価ゲームを開始します..."))
        display(Markdown(f"### **キーワード:** {keyword}"))

        for i in range(60):  # 最大60回のターンを実行
            obs = MockObservation(step, role, turnType, keyword, category, questions, answers, guesses)  # モックオブザベーションを作成

            start_time = time.time()  # 開始時間を記録
            response = agent_fn(obs, None)  # エージェント関数を呼び出す
            end_time = time.time()  # 終了時間を記録

            response_time = end_time - start_time  # 応答時間を計算
            if response_time > 60:
                display(Markdown(f"**WARNING:** 応答時間が長すぎで、ゲームから失格になる可能性があります: {response_time:.2f} 秒。右側のパネルでセッションオプションからGPUアクセラレーションが有効になっていることを確認してください。"))
                break  # 60秒を超えたらループを終了

            # 適切なリストに応答を記録
            if turnType == 'ask':
                questions.append(response)  # 質問を記録
                turnType = 'answer'  # 次は回答
            elif turnType == 'answer':
                answers.append(response)  # 回答を記録
                turnType = 'guess'  # 次は推測
            elif turnType == 'guess':
                guesses.append(response)  # 推測を記録
                if response.lower() == keyword.lower():  # 正しいキーワードが推測された場合
                    display(Markdown(f"## **キーワード '{keyword}' が正しく推測されました！ゲームを終了します。**"))
                    break  # ループを終了
                turnType = 'ask'  # 次は質問
                step += 1  # ステップを増加

            display(Markdown(f"ステップ {step} | 応答: {response} | {response_time:.2f} 秒"))
        display(Markdown(f"最終質問: {', '.join(questions)}"))
        display(Markdown(f"最終回答: {', '.join(answers)}"))
        display(Markdown(f"最終推測: {', '.join(guesses)}"))

    # テストを実行
    test_20_questions()

---

# コメント

> ## Pang Luo
> 
> 本当に印象的な作品です。「考えて答える」アプローチのアイデアは素晴らしいです。エージェントは「ダニューブ」を推測しましたが、正解は「ドニエプル」でした。本当に惜しいですね。

---

> ## Thuận Đoàn Vũ
> 
> あなたの素晴らしい作業に感謝します。ちょっとした質問があるのですが、なぜ4ビットではなく8ビットでモデルをロードしないのですか？8ビットの方が良いのでは？

> 
> > ## Matthew S Farmer (トピック著者)
> > 
> > 素晴らしい洞察ですね。試してみてください！このノートブックはモジュール形式で構築されているので、改善を加えてテストすることができます。🤗
> > 
> > 
> > 
> > ## Matthew S Farmer (トピック著者)
> > 
> > あなたの質問に技術的に答えると、精度が低いほど性能が低下するという意見があります。これが小型モデルでは大きな影響を与えないこともありますが、依然として存在しています。しかし、この競技では計算の制約と応答ごとの時間制限もあります。テストするときは、割り当てられたVRAMと応答時間の制約を守りながら、モデルの能力を最大限に引き出すことを忘れないでください。 
> > 
> > 
> 

---

> ## Marília Prata
> 
> 素晴らしいノートブックと、あなたの貴重なヒント（改善の余地、FAQ、マークダウンセル）に感謝します。

> 
> > ## Matthew S Farmer (トピック著者)
> > 
> > あなたからの高い評価は大変光栄です、マリリア。ありがとうございます！
> > 
> > 
> 

---